In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [2]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [3]:
# Store the CSV you saved created in a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Utrecht,NL,2020-02-24 05:52:38,52.00,5.25,43.0,86,90,6.93,overcast clouds,0.0,0.000000
1,1,Neiafu,TO,2020-02-24 05:55:24,-18.65,-173.98,84.2,79,20,9.17,few clouds,0.0,0.000000
2,2,Klaksvik,FO,2020-02-24 05:55:25,62.23,-6.59,39.2,69,80,17.22,light snow,0.0,0.014961
3,3,Aklavik,CA,2020-02-24 05:55:25,68.22,-135.01,-4.0,76,90,2.24,light snow,0.0,0.000000
4,4,Quesnel,CA,2020-02-24 05:55:25,53.00,-122.50,28.4,100,90,3.36,mist,0.0,0.000000


In [4]:
# Ask the customer
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_bool = input("Do you want it to be raining? (yes/no)")
snow_bool = input("Do you want it to be snowing? (yes/no)")

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no)yes
Do you want it to be snowing? (yes/no)no


In [5]:
if rain_bool == "yes" and snow_bool == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                           (city_data_df["Max Temp"] >= min_temp) & 
                                           (city_data_df["Rain inches (last 3 hrs)"] > 0) & 
                                           (city_data_df["Snow inches (last 3 hrs)"] > 0)]
elif rain_bool == "yes" and snow_bool == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                           (city_data_df["Max Temp"] >= min_temp) & 
                                           (city_data_df["Rain inches (last 3 hrs)"] > 0) & 
                                           (city_data_df["Snow inches (last 3 hrs)"] == 0)]
elif rain_bool == "no" and snow_bool == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                           (city_data_df["Max Temp"] >= min_temp) & 
                                           (city_data_df["Rain inches (last 3 hrs)"] == 0) & 
                                           (city_data_df["Snow inches (last 3 hrs)"] > 0)]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                           (city_data_df["Max Temp"] >= min_temp) & 
                                           (city_data_df["Rain inches (last 3 hrs)"] == 0) & 
                                           (city_data_df["Snow inches (last 3 hrs)"] == 0)]

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Avarua,CK,78.80,light rain,-21.21,-159.78,
44,Butaritari,KI,82.06,moderate rain,3.07,172.79,
91,Pemangkat,ID,84.88,light rain,1.17,108.97,
101,Salinopolis,BR,81.63,light rain,-0.61,-47.36,
117,Faanui,PF,83.30,light rain,-16.48,-151.75,
155,Kribi,CM,79.00,light rain,2.95,9.92,
190,Pandan,PH,81.77,light rain,14.05,124.17,
193,Kindu,CD,72.32,light rain,-2.95,25.95,
203,Dingle,PH,84.43,light rain,11.00,122.67,
208,Geraldton,AU,84.20,light rain,-28.77,114.60,


In [7]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [8]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"

# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")